# Decision Tree Classifier Implementation in Python

#### Importing the required modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

#### Defining Utility Functions for Entropy and GINI calculation

In [2]:
def calcEntropy(pop, target):
    counts = target.value_counts(normalize = True)
    counts *= np.log2(counts)
    entropy = -counts.sum()
    return entropy

In [3]:
def calcGini(pop, target):
    counts = target.value_counts(normalize = True)
    counts = np.square(counts)
    gini = 1 - np.sum(counts)
    return gini

#### Defining Utility Functions to Generate Subpopulations and Find the Optimal Split

In [4]:
def createSubpopulation(pop, target, feature, feature_val):
    return (pop[pop[feature] == feature_val], target[pop[feature] == feature_val])

In [5]:
def minEntropySplit(pop, target, features):
    minfeature = -1
    classes = target.value_counts()
    minEntropy = np.log2(len(classes)) + 1 #maximum possible entropy + 1
    n = len(pop)
    for f in features:
        # split using this feature and calculate entropy
        unique_vals = set(pop.loc[:,f])
        totalEntropy = 0
        for u in unique_vals:
            # split where pop[f] = u
            subpop, subtarget = createSubpopulation(pop, target, f, u)
            ni = len(subpop)
            entropy = calcEntropy(subpop, subtarget)
            totalEntropy += (ni/n)*entropy
        if(totalEntropy < minEntropy):
            minfeature = f
            minEntropy = totalEntropy
    return (minfeature, minEntropy)


In [6]:
def minGiniSplit(pop, target, features):
    minfeature = -1
    classes = target.value_counts()
    minGini = 1 #maximum possible gini (for n-> inf)
    n = len(pop)
    for f in features:
        # split using this feature and calculate Gini
        unique_vals = set(pop.loc[:, f])
        totalGini = 0
        for u in unique_vales:
            #split where pop[j][f] = u
            subpop, subtarget = createSubpopulation(pop, target, f, u)
            ni = len(subpop)
            gini = calcGini(subpop, subtarget)
            totalGini += (ni/n)*gini
        if(totalGini < minGini):
            minGini = totalGini
            minfeature = f
    return (minfeature, minGini)

#### Defining the data

In [7]:
X = pd.DataFrame({
    'outlook': ['sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain', 'overcast', 'sunny', 'sunny', 'rain', 'sunny', 'overcast', 'overcast', 'rain'],
    'temp':['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild'],
    'humidity':['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high'],
    'wind':['strong', 'strong', 'weak', 'weak', 'weak', 'strong', 'weak', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'strong']
})
y = pd.Series([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0], name = "Play tennis")

In [8]:
X.head()

,outlook,temp,humidity,wind
0,sunny,hot,high,strong
1,sunny,hot,high,strong
2,overcast,hot,high,weak
3,rain,mild,high,weak
4,rain,cool,normal,weak


In [9]:
y.head()

0    0
1    0
2    1
3    1
4    1
Name: Play tennis, dtype: int64

#### Generating the training and testing sets from the data
The data is split into features and lables and 30% of the data is used as a testing set and 70% is used as training set

In [10]:
from sklearn.model_selection import train_test_split
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [11]:
X_train.head()

,outlook,temp,humidity,wind
8,sunny,cool,normal,weak
2,overcast,hot,high,weak
1,sunny,hot,high,strong
13,rain,mild,high,strong
4,rain,cool,normal,weak


In [12]:
X_test.head()

,outlook,temp,humidity,wind
9,rain,mild,normal,weak
11,overcast,mild,high,strong
0,sunny,hot,high,strong
12,overcast,hot,normal,weak
5,rain,cool,normal,strong


In [13]:
y_train.head()

8     1
2     1
1     0
13    0
4     1
Name: Play tennis, dtype: int64

In [14]:
y_test.head()

9     1
11    1
0     0
12    1
5     0
Name: Play tennis, dtype: int64

In [15]:
## Information Gain and gini index value of train dataset
calcEntropy(X_train, y_train), calcGini(X_train, y_train)

(0.9182958340544896, 0.4444444444444444)

In [16]:
### Information Gain and gini index value of test dataset
calcEntropy(X_test, y_test), calcGini(X_test, y_test)

(0.9709505944546686, 0.48)

## Implementation of Decision Tree Class without Pruning

In [17]:
class DecisionTree:

        def __init__(self, pop, target, criteria = 'entropy'):
            self.pop = pop
            self.target = target
            self.criteria = criteria
            self.children = {}
            self.sel_feature = None
            self.decision = None
            self.isLeaf = False
            if(criteria == 'entropy'): 
                self.dec_param = calcEntropy(self.pop, self.target)
            elif(criteria == 'gini'): 
                self.dec_param = calcGini(self.pop, self.target)

        def fit(self, features):

        # if the decision parameter of self is 0, make a decision, rather than learning
            if(self.dec_param == 0):
                # already a pure population, take the most popular decision
                self.isLeaf = True
                self.decision = self.target.value_counts().index[0]
                return

         # find the optimal split
            if(self.criteria == 'entropy'):
                f, dec_param = minEntropySplit(self.pop, self.target, features)
            elif(self.criteria == 'gini'):
                f, dec_param = minGiniSplit(self, pop, self.target, features)

         # remove the selected feature from the list of remaining features
            features.remove(f)
            self.sel_feature = f

            unique_vals = set(self.pop.loc[:, f])
            for u in unique_vals:
                #for each possible value of the feature
                #create a child node
                subpop, subtarget = createSubpopulation(self.pop, self.target, self.sel_feature, u)
                self.children[u] = DecisionTree(subpop, subtarget, self.criteria)
                #train that child
                self.children[u].fit(features)

        def predict_row(self, row):
            if(self.isLeaf):
                return self.decision
            else:
                return self.children[row.loc[self.sel_feature]].predict_row(row)

        def predict(self, test_pop):
            preds = []
            for i in range(len(test_pop)):
                preds.append(self.predict_row(test_pop.iloc[i,:]))
            return pd.Series(data = preds, name="preds", index = test_pop.index)

        
        def visualize(self, level=0):
            if(level == 0): print("root ", end = "")
            print("-> ", end = "")
            if(self.isLeaf):
                print("~"+str(self.decision)+"~", end = "")
                print("")
                return
            print(self.sel_feature)
            for i in self.children.keys():
                print("\t"*(2*level+1), "==", i, end = " ")
                self.children[i].visualize(level+1)
            print("")

            

In [18]:
dt = DecisionTree(X_train, y_train)
dt.fit(list(X_train.columns))

In [19]:
dt.visualize()

root -> humidity
	 == high -> outlook
			 == sunny -> ~0~
			 == rain -> wind
					 == strong -> ~0~
					 == weak -> ~1~

			 == overcast -> ~1~

	 == normal -> ~1~



In [20]:
dt.predict(X_test)

9     1
11    1
0     0
12    1
5     1
Name: preds, dtype: int64

In [21]:
y_test

9     1
11    1
0     0
12    1
5     0
Name: Play tennis, dtype: int64

As we clearly see index no. 5 give wrong prediction output.